In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import zipfile
         
fantasy_zip = zipfile.ZipFile('./plates.zip')
fantasy_zip.extractall('./')
fantasy_zip.close()
# import shutil 
# from tqdm import tqdm

# train_dir = 'train'
# val_dir = 'val'

# class_names = ['cleaned', 'dirty']

# for dir_name in [train_dir, val_dir]:
#     for class_name in class_names:
#         os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

# for class_name in class_names:
#     source_dir = os.path.join('./plates', 'train', class_name)
#     for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
#         if i % 6 != 0:
#             dest_dir = os.path.join(train_dir, class_name) 
#         else:
#             dest_dir = os.path.join(val_dir, class_name)
#         shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))


In [0]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import applications, optimizers

In [0]:
img_size = 224
batch_size = 32

In [0]:
%%time
train_datagen=ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True,
        )

train_generator = train_datagen.flow_from_directory(
        './plates/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='binary')

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(  
        './plates',
        classes=['test'],
        target_size = (img_size, img_size),
        batch_size = 1,
        shuffle = False,        
        class_mode = None)        

Found 40 images belonging to 2 classes.
Found 744 images belonging to 1 classes.
CPU times: user 17.9 ms, sys: 2.68 ms, total: 20.6 ms
Wall time: 204 ms


In [1]:
base_model = applications.InceptionResNetV2(weights='imagenet', 
                          include_top=False, 
                          input_shape=(img_size, img_size, 3))
base_model.trainable = False
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)

predictions = Dense(1, activation="sigmoid")(x)
model = Model(input = base_model.input, output = predictions)

model.compile(loss='binary_crossentropy', optimizer = optimizers.rmsprop(lr=0.0001, decay=1e-6), metrics=['accuracy'])

NameError: ignored

In [0]:
%%time
model.fit_generator(
        train_generator,
        steps_per_epoch=500,
        epochs=1,
        verbose=1)

Epoch 1/1
500/500 [==============================] - 332s 665ms/step - loss: 0.2481 - acc: 0.9235
CPU times: user 7min 43s, sys: 50.7 s, total: 8min 33s
Wall time: 5min 45s


In [0]:
%%time
test_generator.reset()
predict = model.predict_generator(test_generator, steps = len(test_generator.filenames))
len(predict)

CPU times: user 32.3 s, sys: 606 ms, total: 32.9 s
Wall time: 32.1 s


In [0]:
sub_df = pd.read_csv('sample_submission.csv')
sub_df.head()
sub_df['label'] = predict
sub_df['label'] = sub_df['label'].apply(lambda x: 'dirty' if x > 0.5 else 'cleaned')
sub_df


,id,label
0,0,dirty
1,1,dirty
2,2,dirty
3,3,dirty
4,4,dirty
...,...,...
739,739,dirty
740,740,dirty
741,741,cleaned
742,742,dirty


In [0]:
sub_df['label'].value_counts()


dirty      635
cleaned    109
Name: label, dtype: int64

In [0]:
sub_df.to_csv('sub.csv', index=False)

dirty    744
Name: label, dtype: int64